## Description

A tutorial to give an example on how to use AMIRAL and get the esitmated object. In here, we use a simulated image of VESTA as an example.

#### Package required for AMIRAL: 
- numpy
- matplotlib
- astropy
- maoppy --> but I need to think how to implement it because it is being set a bit differently
- decovbench --> 
- cython 

To implement the environment, import the environment from .yml file. (Check to see if it is the most-up-to-date version.)

In [ ]:
# Packages required
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits
import os
#Change to your path
os.chdir("/Users/alau/Repo/amiral")
from amiral import instructment, utils, parameter, gradient, minimisation, array
from scipy.optimize import minimize 
%matplotlib inline

In [ ]:
# Global variable
# Parameters to modify
FLUX = 5e8         # Object total intensity [e-]
test_data_dir  = "/Users/alau/Data/amiral_fits/VESTA/"
image_name = "image_noise_100.fits"

To setup the PSF estimation, you first need to:
- a defintion of an ao system
- PSF parameter
- image you would like to perform estimation

In [ ]:
# input variables

aosys_dict = {
    'diameter': 7 , 
    'occ_ratio': 0.1 , 
    'no_acutuator' : 30, 
    'wavelength': 500, 
    'dimension': 256,
    'resolution_rad' : 3.5e-8 
}

In [ ]:
# input variables

amiral_dict = {
    "r0": 0.5,  #0.2                
    "background": 0.01,      
    "amplitude": 0.1,  #1.6     
    "ax": 0.05,                            
    "beta": 1.5, 
    "mu": 0., 
    "rho0": 0., 
    "p": 0. 
}

In [ ]:
# What variables to be minimised
param_mask = np.asarray([1,0,0,0,0])
hyper_param_mask = np.asarray([0,0,0])

mask = np.concatenate((param_mask,hyper_param_mask))



As you can see in here, PSF hyperparameters are not initialised. Therefore, it will be dealt with later on. Now convert the dict into arrays

In [ ]:
psf_keys, psf_guess = utils.dict2array(amiral_dict)

In [ ]:
img = utils.load_fits(test_data_dir+image_name)

In [ ]:
aosys_cls = instructment.aoSystem( 
        diameter = aosys_dict['diameter'], 
        occ_ratio = aosys_dict['occ_ratio'], 
        no_acutuator= aosys_dict['no_acutuator'], 
        wavelength = aosys_dict['wavelength']*1e-9, 
        resolution_rad = aosys_dict['resolution_rad'], 
        dimension=aosys_dict['dimension'])  


print(aosys_cls.N_padded)


amiralparam = parameter.amiralParam(img ,guess = psf_guess, aosys = aosys_cls)

Now, we need to set up the bounds, hyperparameters and the fourier variables before calculating the criterion. 

In [ ]:
hyper_guess = amiralparam.hyperparam_initial(psf_guess)
hyper_min, hyper_max = amiralparam.hyperparam_bound(psf_guess, p_upperbound = 100.)

psf_guess[-3] = hyper_guess[0] 
psf_guess[-2] = hyper_guess[1]
psf_guess[-1] = hyper_guess[2]

param_min = np.asarray([0.01,0,0,1e-8,1.01])
param_max =  np.asarray([1.,1e8,1e8,1e3,10])

upperbound = np.concatenate((param_max, hyper_max))
lowerbound = np.concatenate((param_min, hyper_min))

param_numerical_condition = np.array([1., 1e-4, 1., 1., 1.])
hyperparam_numerical_condition = np.array([hyper_guess[0], hyper_guess[1], 1.])

numerical_condition = np.concatenate((param_numerical_condition, hyperparam_numerical_condition))

amiral_cls = parameter.amiral(img=img, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)


In [ ]:
numeric_param = minimisation.param_physical2numerical(psf_guess, amiral_cls.numerical_condition)

In [ ]:
print(psf_guess)
amiral_cls.varible4criterion(psf_guess)
hyper_guess = amiralparam.hyperparam_initial(psf_guess)

In [ ]:
crit = amiral_cls.marg_criterion(psf_guess)
amiral_cls.gradient(psf_guess)
print(crit)

In [ ]:
est_criterion, value_criterion, value_grad = amiral_cls.minimisation(psf_guess)

In [ ]:
print(est_criterion)

In [ ]:
est_criterion_1, value_criterion, value_grad = amiral_cls.minimisation(est_criterion)

In [ ]:
psf_guess

In [ ]:
# grad_IDL = np.array([755510.03424863936    
#                      ,-588575.62182685791      
#                      ,-22049.077696321303      
#                      ,-29376.357243423521       
#                      ,2323.1575165394297
#                      ,-4379524002998.0815       
#                      ,30821.644951204078      
#                      ,-78541.771455431299])

In [ ]:
grad_norm = amiral_cls.gradient(psf_guess)
print(grad_norm)

In [ ]:
k_IDL = 1.1656234695454554e+17
k_py = 1.1651576e+17


err = 100 * (k_py-k_IDL)/k_IDL
print("K hat error in %: ", err, k_py-k_IDL)

In [ ]:
1 - (k_py/k_IDL) 

In [ ]:
# dJdO_IDL_path = "/Users/alau/dJdo.fits"
# dJdo_IDL_f = fits.open(dJdO_IDL_path)

# dJdo_IDL = dJdo_IDL_f[0].data

To check dJdo, you actually need to calculate the fourier variables again, because the old one carries information in the minimisers

In [ ]:
# # dJdo = .5 * (amiral_cls.fourier_variable["psd(psf)"]/amiral_cls.fourier_variable["psd_model_i"]-
# #             (amiral_cls.fourier_variable["psd(psf)"]*amiral_cls.fourier_variable["error2"])/
# #             (amiral_cls.fourier_variable["k_hat"]*(amiral_cls.fourier_variable["psd_model_i"]**2)))

# print(psf_guess)

# amiral_test = parameter.amiral(img=img, guess=psf_guess, aosys = aosys_cls, upperbound = upperbound, lowerbound= lowerbound, numerical_condition = numerical_condition, fourier_variable = amiralparam.fourier_variable, mask = mask)

# fourier_cal = amiral_test.varible4criterion(psf_guess, debug = True)



# dJdo = .5 * (amiral_test.fourier_variable["psd(psf)"]/amiral_test.fourier_variable["psd_model_i"]-
#             (amiral_test.fourier_variable["psd(psf)"]*amiral_test.fourier_variable["error2"])/
#             (amiral_test.fourier_variable["k_hat"]*(amiral_test.fourier_variable["psd_model_i"]**2)))

In [ ]:
# amiral_test.fourier_variable["k_hat"]

In [ ]:
# plt.imshow(dJdo)

In [ ]:
# plt.imshow(np.fft.fftshift(dJdo_IDL))

In [ ]:
# diff = utils.view_diff(np.fft.fftshift(dJdo_IDL),dJdo)

In [ ]:
# utils.info(dJdo)

In [ ]:
# np.sum(diff)
# utils.info(diff)

In [ ]:
# np.sum(dJdo)

In [ ]:
# np.sum(dJdo_IDL)

In [ ]:
100*((3.56459663387992e-9 - 3.5664025e-09)/3.5664025e-09)

In [ ]:
# -4.9398799e+12       26422.690      -65779.984
grad_hyper_py = [-4.94448401e+12, 2.64339184e+04, -6.58094955e+04]
grad_hyper_IDL = [-4.9398799e+12,26422.690, -65779.984]

print(100*(grad_hyper_py[0]-grad_hyper_IDL[0])/grad_hyper_IDL[0], 
      100*(grad_hyper_py[1]-grad_hyper_IDL[1])/grad_hyper_IDL[1], 
     100*(grad_hyper_py[2]-grad_hyper_IDL[2])/grad_hyper_IDL[2])

In [ ]:
print("Numerical condition")
print(numerical_condition)

In [ ]:
a = np.zeros((4,2))

for i in range (4):
    a[i][0] = i
    a[i][0] = i+1

In [ ]:
a